# Exercise - Forward Pricing and Carry

#### Notation Commands

$$\newcommand{\Black}{\mathcal{B}}
\newcommand{\Blackcall}{\Black_{\mathrm{call}}}
\newcommand{\Blackput}{\Black_{\mathrm{put}}}
\newcommand{\EcondS}{\hat{S}_{\mathrm{conditional}}}
\newcommand{\Efwd}{\mathbb{E}^{T}}
\newcommand{\Ern}{\mathbb{E}^{\mathbb{Q}}}
\newcommand{\Tfwd}{T_{\mathrm{fwd}}}
\newcommand{\Tunder}{T_{\mathrm{bond}}}
\newcommand{\accint}{A}
\newcommand{\carry}{\widetilde{\cpn}}
\newcommand{\cashflow}{C}
\newcommand{\convert}{\phi}
\newcommand{\cpn}{c}
\newcommand{\ctd}{\mathrm{CTD}}
\newcommand{\disc}{Z}
\newcommand{\done}{d_{1}}
\newcommand{\dt}{\Delta t}
\newcommand{\dtwo}{d_{2}}
\newcommand{\flatvol}{\sigma_{\mathrm{flat}}}
\newcommand{\flatvolT}{\sigma_{\mathrm{flat},T}}
\newcommand{\float}{\mathrm{flt}}
\newcommand{\freq}{m}
\newcommand{\futprice}{\mathcal{F}(t,T)}
\newcommand{\futpriceDT}{\mathcal{F}(t+h,T)}
\newcommand{\futpriceT}{\mathcal{F}(T,T)}
\newcommand{\futrate}{\mathscr{f}}
\newcommand{\fwdprice}{F(t,T)}
\newcommand{\fwdpriceDT}{F(t+h,T)}
\newcommand{\fwdpriceT}{F(T,T)}
\newcommand{\fwdrate}{f}
\newcommand{\fwdvol}{\sigma_{\mathrm{fwd}}}
\newcommand{\fwdvolTi}{\sigma_{\mathrm{fwd},T_i}}
\newcommand{\grossbasis}{B}
\newcommand{\hedge}{\Delta}
\newcommand{\ivol}{\sigma_{\mathrm{imp}}}
\newcommand{\logprice}{p}
\newcommand{\logyield}{y}
\newcommand{\mat}{(n)}
\newcommand{\nargcond}{d_{1}}
\newcommand{\nargexer}{d_{2}}
\newcommand{\netbasis}{\tilde{\grossbasis}}
\newcommand{\normcdf}{\mathcal{N}}
\newcommand{\notional}{K}
\newcommand{\pfwd}{P_{\mathrm{fwd}}}
\newcommand{\pnl}{\Pi}
\newcommand{\price}{P}
\newcommand{\probexer}{\hat{\mathcal{P}}_{\mathrm{exercise}}}
\newcommand{\pvstrike}{K^*}
\newcommand{\refrate}{r^{\mathrm{ref}}}
\newcommand{\rrepo}{r^{\mathrm{repo}}}
\newcommand{\spotrate}{r}
\newcommand{\spread}{s}
\newcommand{\strike}{K}
\newcommand{\swap}{\mathrm{sw}}
\newcommand{\swaprate}{\cpn_{\swap}}
\newcommand{\tbond}{\mathrm{fix}}
\newcommand{\ttm}{\tau}
\newcommand{\value}{V}
\newcommand{\vega}{\nu}
\newcommand{\years}{\tau}
\newcommand{\yearsACT}{\tau_{\mathrm{act/360}}}
\newcommand{\yield}{Y}$$

---

# 1. Forward Pricing Fundamentals

### Data

Consider a Treasury note with the following characteristics:

| Field | Value |
|-------|-------|
| Coupon | 4.00% |
| Maturity | 2030-02-28 |
| Issue Date | 2023-02-28 |
| First Coupon | 2023-08-31 |

Market conditions as of **2023-04-18**:

| Field | Value |
|-------|-------|
| Spot Price | 102-02 (i.e., 102 + 2/32) |
| Repo Rate | 4.85% |
| Forward Date | 2023-08-01 |

In [ ]:
import pandas as pd
import numpy as np
from datetime import date

In [ ]:
# Bond specifications
date_issue = date(2023, 2, 28)
date_maturity = date(2030, 2, 28)
date_coupon = date(2023, 8, 31)
cpn = 0.04
face = 100

# Market conditions
date_now = date(2023, 4, 18)
date_fwd = date(2023, 8, 1)
spot_price = 102 + 2/32
repo_rate = 0.0485

### 1.1 Accrued Interest

Calculate the accrued interest at:
- Time $t$ (the current date)
- Time $T$ (the forward date)

#### Tip: Accrued Interest Formula

$$\accint_t = \frac{\cpn \cdot N}{2} \times \frac{\text{days since last coupon}}{\text{days in coupon period}}$$

where $N$ is the face value (100).

### 1.2 Forward Price

Calculate the forward price of the bond for delivery at time $T$.

#### Tip: Forward Price Formula (No Interim Coupon)

$$\fwdprice = (P_t + \accint_t)(1 + \rrepo \cdot \yearsACT) - \accint_T$$

where:
- $P_t$ is the clean spot price
- $\accint_t$ is accrued interest at $t$
- $\rrepo$ is the repo rate
- $\yearsACT$ is the forward interval in ACT/360 years
- $\accint_T$ is accrued interest at $T$

### 1.3 The Forward Drop

Calculate the **forward drop**, which is the difference between the spot and forward prices.

Is the forward price higher or lower than the spot? Explain why, in terms of the carry relationship.

---

# 2. Carry Analysis

### 2.1 Calculate Carry

Compute the **carry** of holding this bond from $t$ to $T$.

#### Tip: Carry Formula

$$\text{Carry} = \left(\cpn \cdot N - \rrepo \cdot P_t^{\text{dirty}}\right) \cdot \yearsACT$$

where $P_t^{\text{dirty}} = P_t + \accint_t$.

Alternatively, expressed as a rate:
$$\carry = \cpn - \rrepo$$

### 2.2 Verify the Relationship

Verify that the forward drop approximately equals the carry:

$$P_t - \fwdprice \approx \text{Carry}$$

Why might this relationship not hold exactly?

---

# 3. Forward Pricing with Interim Coupon

Now consider a forward contract that spans across a coupon date.

**New scenario:**
- Current date: 2023-04-18
- Forward date: 2023-10-15 (after the August coupon)
- Coupon payment date: 2023-08-31

The bond still has:
- Coupon: 4%
- Spot price: 102-02
- Repo rate: 4.85%

In [ ]:
# New forward date (after coupon)
date_fwd_new = date(2023, 10, 15)
date_next_coupon = date(2023, 8, 31)

### 3.1 Forward Price with Interim Coupon

Calculate the forward price when there is an interim coupon payment.

#### Tip: Forward Price with Interim Coupon

$$\fwdprice = \left[(P_t + \accint_t)(1 + \rrepo \cdot \tau_0) - \frac{\cpn \cdot N}{2}\right](1 + \rrepo \cdot \tau_1) - \accint_T$$

where:
- $\tau_0$ is the pre-coupon interval (ACT/360)
- $\tau_1$ is the post-coupon interval (ACT/360)

### 3.2 Compare Forward Prices

How does the forward price with an interim coupon compare to a hypothetical forward price calculated without accounting for the coupon? 

Explain the economic intuition.

---

# 4. Forward vs Futures

### 4.1 Daily Settlement Impact

Suppose the Treasury futures contract on this bond is trading at 101.50.

Given your calculated forward price, is the futures price higher or lower than the forward price?

Explain why Treasury futures prices are typically **lower** than forward prices due to the daily settlement (mark-to-market) mechanism.

### 4.2 Tailing the Hedge

If you want to hedge a $10 million notional forward position using futures, how many futures contracts should you use?

#### Tip: Tailing the Hedge

$$N^{\text{fut}} = Z(t,T) \cdot N^{\text{fwd}}$$

where $Z(t,T) = \frac{1}{1 + \rrepo \cdot \yearsACT}$

Assume each futures contract has a notional of $100,000.

***